# Test fit of a single object

Here we look at a single object in detail.

The notebook can be downloaded [here](https://github.com/lephare-photoz/lephare/blob/main/docs/notebooks/Testing_fit_of_one_object_of_the_catalogue.ipynb).

In [ ]:
import os
import lephare as lp
import numpy as np
from matplotlib import pylab as plt

%matplotlib inline

In [ ]:
config = lp.default_cosmos_config.copy()
config.update(
    {
        # "ZPHOTLIB": "VISTA_COSMOS_FREE,ALLSTAR_COSMOS,QSO_COSMOS",
        "ZPHOTLIB": "BC03_COSMOS,ALLSTAR_COSMOS,QSO_COSMOS",
        "CAT_IN": f"{lp.LEPHAREDIR}/examples/COSMOS.in",
        "CAT_OUT": "zphot_short.out",
        "ADD_EMLINES": "0,100",
        "AUTO_ADAPT": "NO",
        "Z_STEP": "0.02,0,1",  # We are looking at a source at z<1
        "CAT_LINES": "1,100",
        "SPEC_OUT": "NO",
        "PARA_OUT": f"{lp.LEPHAREDIR}/examples/output.para",
        "VERBOSE": "NO",
        "ZFIX": "NO",
        "SEL_AGE": "$LEPHAREDIR/sed/GAL/BC03_CHAB/AGE_BC03COMB.dat",
    }
)

In [ ]:
# Get the auxiliary files required.
lp.data_retrieval.get_auxiliary_data(keymap=config, additional_files=["examples/COSMOS.in"])

In [ ]:
# we can write the config to a file to keep a record
config_file = "./config_file.para"
lp.write_para_config(config, config_file)
# before running PhotoZ we must run filt, sedtolib and maggal
filterLib = lp.Filter(config_file=config_file)
filterLib.run()
# Note how native c++ classes require the config in the lephare native format
keymap = lp.all_types_to_keymap(config)
sedlib = lp.Sedtolib(config_keymap=keymap)
sedlib.run(typ="STAR", star_sed="$LEPHAREDIR/sed/STAR/STAR_MOD_ALL.list")
sedlib.run(typ="QSO", qso_sed="$LEPHAREDIR/sed/QSO/SALVATO09/AGN_MOD.list", gal_lib="LIB_QSO")
# sedlib.run(typ="GAL", gal_sed="$LEPHAREDIR/sed/GAL/COSMOS_SED/COSMOS_MOD.list", gal_lib="LIB_GAL")
sedlib.run(
    typ="GAL",
    gal_sed="$LEPHAREDIR/sed/GAL/BC03_CHAB/BC03COMB_MOD.list",
    gal_lib="LIB_GAL",
    sel_age="$LEPHAREDIR/sed/GAL/BC03_CHAB/AGE_BC03COMB.dat",  # Seems not read here. Why?
)

maglib = lp.MagGal(config_keymap=keymap)
maglib.run(typ="STAR", lib_ascii="NO", star_lib_out="ALLSTAR_COSMOS")
maglib.run(
    typ="QSO",
    lib_ascii="NO",
    mod_extinc="0,1000",
    eb_v="0.,0.1,0.2,0.3",
    extinc_law="SB_calzetti.dat",
    qso_lib_in="LIB_QSO",
)
maglib.run(
    typ="GAL",
    lib_ascii="NO",
    gal_lib_in="LIB_GAL",
    # gal_lib_out="VISTA_COSMOS_FREE",
    # mod_extinc="18,26,26,33,26,33,26,33",
    # extinc_law="SMC_prevot.dat,SB_calzetti.dat,SB_calzetti_bump1.dat,SB_calzetti_bump2.dat",
    # em_lines="EMP_UV",
    # em_dispersion="0.5,0.75,1.,1.5,2.",
    gal_lib_out="BC03_COSMOS",
    eb_v="0.,0.1,0.2,0.3,0.4,0.5",
    mod_extinc="0,12",
    extinc_law="SB_calzetti.dat",
    em_lines="PHYS",
    em_dispersion="1.",
)

In [ ]:
photz = lp.PhotoZ(keymap)

In [ ]:
filts = lp.filterSvc.FilterSvc.from_config(config_file)

In [ ]:
# for filt in filts:
#    plt.figure()
#    filt.plot_filter_curve()

In [ ]:
cat = np.loadtxt(f"{lp.LEPHAREDIR}/examples/COSMOS.in")
nfilt = 30
pos = 57
entry = cat[pos]
identifier = str(entry[0])
fluxes = entry[np.arange(1, nfilt * 2 + 1, 2)]
efluxes = entry[np.arange(1, nfilt * 2 + 1, 2) + 1]
context = int(entry[-3])
zspec = entry[-2]
additional_str = str(entry[-1])

In [ ]:
src = lp.onesource(0, photz.gridz)
src.readsource(identifier, fluxes, efluxes, context, zspec, additional_str)

In [ ]:
photz.prep_data(src)

In [ ]:
a0 = photz.compute_offsets([])
src.adapt_mag(a0);

In [ ]:
photz.fit_onesource(src)
print(src.zmin, src.chimin)

In [ ]:
photz.uncertainties_onesource(src)

In [ ]:
photz.physpara_onesource(src)

In [ ]:
src.spec, src.indmin, src.consiz

In [ ]:
# %load_ext autoreload
# %autoreload 2
# import lephare
# from lephare import *
minl = 1.0e10
maxl = 0
for f in filts:
    if minl > f.lmean - 2 * f.dwidth:
        minl = f.lmean - 2 * f.dwidth
    if maxl < f.lmean + 2 * f.dwidth:
        maxl = f.lmean + 2 * f.dwidth
print(minl, maxl)

In [ ]:
gal1 = photz.besttemplate_onesource(src, 0, minl, maxl)
gal2 = photz.besttemplate_onesource(src, 1, minl, maxl)
fir = photz.besttemplate_onesource(src, 2, minl, maxl)
qso = photz.besttemplate_onesource(src, 3, minl, maxl)
star = photz.besttemplate_onesource(src, 4, minl, maxl)

In [ ]:
models_info = []  # à remplir
if src.indmin[0] >= 0:
    models_info.append(
        [
            "GAL-1",
            len(gal1[0]),
            str(src.imasmin[0]),
            "1",
            str(src.nbused),
            str(src.consiz),
            src.zgmin[1],
            src.zgmin[2],
            src.chimin[0],
            "-1",
            # photz.fullLib[src.indmin[0]].extlawId,
            # photz.fullLib[src.indmin[0]].ebv,
            src.Ldustmed[0],
            src.agemed[0],
            src.massmed[0],
            src.SFRmed[0],
            src.sSFRmed[0],
        ]
    )
else:
    models_info.append(["GAL-1 0 -1 -1 -1 -1. -1. -1. -1. -1. -1 -1. -1. -1. -1. -1. -1.".split()])

# second solution
if src.indminSec >= 0:
    models_info.append(
        [
            "GAL-2",
            len(gal2[0]),
            str(src.zsecMod),
            "1",
            str(src.nbused),
            src.zsec,
            "-1",
            "-1",
            src.zsecChi2,
            src.zsecProb,
        ]
        + "-1 -1. -1. -1. -1. -1. -1.".split()
    )
else:
    models_info.append("GAL-2 0 -1 -1 -1 -1. -1. -1. -1. -1. -1 -1. -1. -1. -1. -1. -1.".split())

# Galaxy FIR
if src.indminIR >= 0:
    models_info.append(
        [
            "GAL-FIR",
            len(fir[0]),
            imasminIR,
            "1",
            src.nbused,
            src.zminIR,
            "-1 -1".split(),
            src.chiminIR,
            "0 -1 -1".split(),
            src.LIRml,
        ]
        + "-1 -1 -1 -1".split()
    )
else:
    models_info.append("GAL-FIR 0 -1 -1 -1 -1. -1. -1. -1. -1. -1 -1. -1. -1. -1. -1. -1.".split())

## STOCH
##models_info.append("GAL-STOCH 0 -1 -1 -1 -1. -1. -1. -1. -1. -1 -1. -1. -1. -1. -1. -1.".split())

# QSO
if src.zmin[1] > 0:
    models_info.append(
        [
            "QSO",
            len(qso[0]),
            str(src.imasmin[1]),
            "2",
            str(src.nbused),
            str(src.zmin[1]),
            "0",
            "0",
            src.chimin[1],
        ]
        + "0. -1 -1. -1. -1. -1. -1. -1.".split()
    )

else:
    models_info.append("QSO 0 -1 -1 -1 -1. -1. -1. -1. -1. -1 -1. -1. -1. -1. -1. -1.".split())

# STAR
if src.chimin[2] > 0:
    models_info.append(
        ["STAR", len(star[0]), str(src.imasmin[2]), "3", str(src.nbused), "0", "0", "0", src.chimin[2]]
        + "0. -1 -1. -1. -1. -1. -1. -1.".split()
    )
else:
    models_info.append("STAR 0 -1 -1 -1 -1. -1. -1. -1. -1. -1 -1. -1. -1. -1. -1. -1.".split())

In [ ]:
id = src.spec
zspec = str(src.zs)
zphot = src.consiz
nfilt = len(filts)
npdf = src.pdfmap[lp.maptype["MIN_ZG"]].size()
mag = np.array(src.mab)
em = np.array(src.msab)
lf = np.array([f.lmean * 10000 for f in filts])
dlf = np.array([f.dwidth * 10000 for f in filts])
zpdf = np.zeros((3, npdf))
zpdf[0, :] = src.pdfmap[lp.maptype["BAY_ZG"]].xaxis
zpdf[1, :] = src.pdfmap[lp.maptype["BAY_ZG"]].vPDF
zpdf[2, :] = src.pdfmap[lp.maptype["MIN_ZG"]].vPDF
lg = []
mg = []
for sol in [gal1, gal2, fir, qso, star]:
    lg.append(np.array(sol[0]))
    mg.append(-0.4 * (np.array(sol[1]) - 23.91))  # logFnu

In [ ]:
##############  PLOT  ############### FROM SPEC.PY

### Initialise the figure
fig = plt.figure()

### Main panel
ax1 = fig.add_axes(
    [0.1, 0.1, 0.78, 0.78], xscale="log", xlabel="$\lambda$ [$\mu$m]", ylabel="log(F$_{\\nu}$) [$\mu$Jy]"
)

# only the reliable obs mag will be plotted:
em = em * 2.0
dlf = dlf / 2.0
mag1 = mag[(mag > 0.0) & (mag < 35) & (em > -3)]
em1 = em[(mag > 0.0) & (mag < 35) & (em > -3)]
lf1 = lf[(mag > 0.0) & (mag < 35) & (em > -3)] / 10000.0
dlf1 = dlf[(mag > 0.0) & (mag < 35) & (em > -3)] / 10000.0

if len(mag1 > 0):
    ymin = max(mag1 + 2.0)
    ymax = min(mag1 - 4.0)
else:
    ymin = 10
    ymax = 20
if ymin > 60:
    ymin = 30

ic = (em1 >= 0.0) & (em1 < 2.0)
lf2 = lf1[ic]
mag2 = -0.4 * (mag1[ic] - 23.91)
em2 = 0.4 * em1[ic]
dlf2 = dlf1[ic]
# low S/N bands:
ic2 = (em1 >= 2.0) & (em1 < 8.0)
lf2b = lf1[ic2]
mag2b = -0.4 * (mag1[ic2] - 23.91)
em2b = 0.4 * em1[ic2]
dlf2b = dlf1[ic2]

print(em)
# set the plot aspect
if len(lf1 > 0):
    ax1.axis([min(lf1) * 0.85, max(lf1) * 1.2, -0.4 * (ymin - 23.91), -0.4 * (ymax - 23.91)])
else:
    ax1.axis([0, 100000, -0.4 * (ymin - 23.91), -0.4 * (ymax - 23.91)])
### plot SED and print info of best-fit models
col_lst = ["r", "g", "b", "m", "y"]  # each one with a different color
plt.figtext(
    0.15,
    0.96,
    " Type: (Model Library Nband) z_phot  Chi^2,  Extlaw  EB-V  Lir  Age  logM*  logSFR",
    size="small",
)
plt.figtext(0.73, 0.12, "ID=" + id, size="small")
iml = 0
for im in range(len(models_info)):
    if int(models_info[im][2]) == -1:
        continue  # print only models really used
    iml = iml + 1  # counter of models used
    ax1.plot(lg[im], mg[im], color=col_lst[im])  # plot the SED
    del models_info[im][6:8]  # do not print z_inf and z_sup
    del models_info[im][-1]  # nor sSFR
    info1 = ("  ".join(["%.3f"] * len(models_info[im][5:7]))) % tuple(
        [float(j) for j in models_info[im][5:7]]
    )
    if float(models_info[im][8]) >= 0.0:  # additional information
        info2 = ("   ".join(["%.2f"] * len(models_info[im][8:]))) % tuple(
            [float(j) for j in models_info[im][8:]]
        )
        info2 = ",  " + info2 + "."
    else:
        info2 = "."
    infol = models_info[im][0] + ": (" + " ".join(models_info[im][2:5]) + ")  " + info1 + info2
    plt.figtext(0.15, 0.96 - 0.02 * iml, infol, color=col_lst[im], size="x-small")  # print the rest

# plot the obs mag...
ax1.errorbar(lf2b, mag2b, yerr=em2b, xerr=dlf2b, fmt="o", color="0.6")
ax1.errorbar(lf2, mag2, yerr=em2, xerr=dlf2, fmt="o", color="0.")
# ... and upper limits
iu = np.where(em1 < 0)
if len(iu[0]) > 0:
    lf3 = lf1[iu]
    mag3 = -0.4 * (mag1[iu] - 23.91)
    ax1.quiver(lf3, mag3, 0, -1, units="height", width=0.004, headwidth=5, color="k", pivot="tip")

### 2nd panel (inset) showing PDF(z)
base = 0.9 - 0.02 * iml  # starting position for the inset plot
if base > 0.84:
    base = 0.84
ax2 = fig.add_axes([0.13, base - 0.20, 0.3, 0.20], xlabel="z_phot", title="z_spec=" + zspec)
ax2.yaxis.set_label_position("right")
ax2.yaxis.set_ticks_position("right")
ax2.plot(zpdf[0, :], zpdf[1, :] / max(zpdf[1, :]), color="r")
ax2.plot(zpdf[0, :], zpdf[2, :] / max(zpdf[2, :]), color="b")
# plot also z_phot with error bar
# ax2.errorbar(zphot,0.5,fmt='ok',xerr=[[zphot-z68low],[z68hig-zphot]],mfc='none')

In [ ]:
zarr = src.pdfmap[11].xaxis
zgmin = src.pdfmap[lp.maptype["MIN_ZG"]].vPDF
zgbay = src.pdfmap[lp.maptype["BAY_ZG"]].vPDF
fig, ax1 = plt.subplots()
ax1.set_title("Source Id: " + src.spec)
color = "tab:red"
ax1.set_xlabel("z")
ax1.set_ylabel("marginalized pdf", color=color)
plt.plot(zarr, zgbay, color=color)
ax1.tick_params(axis="y", labelcolor=color)
ax2 = ax1.twinx()
color = "tab:blue"
ax2.set_ylabel("chi2", color=color)  # we already handled the x-label with ax1
ax2.plot(zarr, -2 * np.log(zgmin), color=color)
ax2.tick_params(axis="y", labelcolor=color)
ax2.set_xlim(src.zgmin[0] - src.zgmin[5], src.zgmin[0] + src.zgmin[6])
fig.tight_layout()

In [ ]:
mask = np.array(src.pdfmap[10].chi2) < 1.0e9
plt.plot(np.array(zarr)[mask], np.array(src.pdfmap[10].chi2)[mask])
plt.xlim(0.0, 0.6)
# plt.ylim(0, 2000)